# Concept Notebook
Start with a blank default application and programmatically add views and data to those views

### Use Case
Programmatically load a single FITS file, e.g., for 2d spectroscopic data, into an "image" viewer.  Or a 1d spectrum into a spectrum viewer.  For cases that do not fit into a given pre-made configuration. 

### MAST Use Case
MAST auto-generates notebooks that when users run, needs to download the data, create the relevant jdaviz application / viewers, and load the data by default. One click run gets the user back to where they were on the web.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [ ]:
from jdaviz.app import Application

In [ ]:
# some 2d spectroscopic data
data = '/Users/bcherinka/Work/jwst/test/jw00626-o030_s00000_nirspec_f170lp-g235m_s2d.fits'

### Option 1 - Create a default viewer with the UI and programmatically add data
In this option, we create jdaviz with a default configuration, and load an "Image 2D" viewer via the user interface, (viewer creator plugin).  We then attempt to add data to the viewer programmatically.  

In [ ]:
app = Application('default')
app

I know I first need to load the data

In [ ]:
app.load_data(data)

# running this twice loads the same data with the same labels.  Downstream analysis/plugin calls raise errors on duplicate and 
# ambiguous data labels.  Running this twice on the same data should either not re-load it or load it with a different unique
# data label

In [ ]:
# what label did my data get?  dig around in docs to understand data_collection.  How do I define custom data labels?
app.data_collection

# want - option to easily label or relabel incoming data.  Either as input in app.load_data or a new function, e.g., app.relabel_data()
# app.add_data requires data to be in a special format

In [ ]:
# get a label
label = app.data_collection[0].label
label

I want to load the data into the viewer.  I don't know the name of the viewer I created.  How do I get a list of current viewer references?

In [ ]:
# naively try to get "image-viewer". ; throws error
app.get_viewer('image-viewer')

# also try 
#app.get_viewer('g-image-viewer')
#app.get_viewer('flux-viewer')

# want - to return a list of all viewer reference names for currently loaded viewers.  Pick from the list and add data.
# app.list_viewers()

In [ ]:
# let's try to add data anyways - throws same error
app.add_data_to_viewer('image-viewer', data, ext='SCI')

How can I access and inspect the available viewers?  

In [ ]:
# look in the viewer registry
from jdaviz.core.registries import tool_registry, viewer_registry
viewer_registry

In [ ]:
# I can find all viewers in the registry. But these dict keys are not reference names.  
viewer_registry.members

In [ ]:
# oh I see my "Image 2D" == "g-image-viewer".  That must be the reference name.  Let me try that, to get the viewer.
app.get_viewer('g-image-viewer')

Now I'm lost and don't know how to look up my viewer.  I can dig around in hidden attributes.

In [ ]:
# look up the viewer store.  I see my viewer with a viewer id. 
app._viewer_store

In [ ]:
# examine the application stack of items
app.state.stack_items

In [ ]:
# loop over all the items in the application stack and print the data
for i in app.state.stack_items:
    print('item', i.keys())
    for viewer in i.get('viewers'):
        print('viewer', viewer.keys())
        for kk, vv in viewer.items():
            print(kk,vv)

In [ ]:
# now I have the viewer, I think.  The repr here should be improved to more obvious.
viewer

In [ ]:
# get the viewer by id
id = viewer['id']

In [ ]:
# try to add data by id.  Still same error as above.
app.add_data_to_viewer(id, label)

In [ ]:
# I notice in the stack that no reference name is indicated.  Let's add one.
view = app._viewer_item_by_id(id)
view['reference'] = 'image-viewer'
view

In [ ]:
# now I can add data to the viewer.  Success.
app.add_data_to_viewer('image-viewer', data, ext='SCI')

In [ ]:
# can I access the data back out? 
app.get_viewer('image-viewer').data()

### Option 2 - Create the viewer programmatically and also load some data
In this option, we create a blank jdaviz application with default configuration, and attempt to create a "Image 2D" viewer programmatically, and add data to it. 

In [ ]:
app = Application()
app

I don't know what viewers are available to add.  How can I find this out?

In [ ]:
# how do I know what viewers are available to add?  Use the viewer registry I dug around in the code to find.
viewer_registry.members

# want - option to list/dict the available options for creating different viewers; returns a reference name, label and class.
# i.e. expose the registry members in a user-friendly format
# app.list_viewer_types()

ok, so I know I want an Image 2d viewer, but don't know how to, or cannot, create one programmatically.

In [ ]:
# want - option to create a viewer by reference name or label.  The input should check against the formatted list from the registry.
# app.create_viewer('image-viewer') or app.create_viewer('Image 2D')

# hack to create a new viewer programmatically using code from the Viewer Creator plugin
from jdaviz.core.events import NewViewerMessage
viewer_cls = viewer_registry.members['g-image-viewer']['cls']
new_viewer_message = NewViewerMessage(viewer_cls, data=None, sender=app)
app.hub.broadcast(new_viewer_message)

In [ ]:
# ok, I created a viewer.  Now have same problem as option 1
app.load_data(data)

In [ ]:
# look for the viewer in the store
app._viewer_store

In [ ]:
# attempt to add the data with naive reference name
app.add_data_to_viewer('image-viewer', data, ext='SCI') 

In [ ]:
# grab the viewer information
id = next(iter(app._viewer_store))
view = app._viewer_item_by_id(id)
for k, v in view.items():
    print(k,v)

In [ ]:
# add valid reference name
view['reference'] = 'image-viewer'

In [ ]:
# re-attempt to add the data.  success
app.add_data_to_viewer('image-viewer', data, ext='SCI') 

### Option 3 - Create custom Helper and define a custom configuration
This option attempts to create a custom helper that represents my 2d spectroscopic fits file, with a single "Image 2D viewer".  I define a new custom configuration with the viewer I need.

In [ ]:
from jdaviz.core.helpers import ConfigHelper
from jdaviz.core.config import get_configuration

# get a default configuration
config = get_configuration('default')

# create a viewer area and viewer
config['viewer_area'] = [{'container': 'col', 'children': [{'container':'row', 'viewers':[]}]}]
viewers = {'viewers': [{'name': 'Image', 'plot': 'g-image-viewer', 'reference': 'image-viewer'}]}

# update the default config
config['viewer_area'][0]['children'][0].update(viewers)

config


# want - easier mechanism for adding viewers into a custom configuration.  Given a custom config, and a viewer 
# reference name or label, return a properly formatted new config
# add_viewer_to_config(config, 'image-viewer')  or add_viewer_to_config(config, ['image-viewer', 'spectrum-viewer'])

In [ ]:
# set my custom config object to the default configuration.  I could also just use Application(config) so 
# what does the ConfigHelper gain me, the user?  Would be nice to have default data loading. 
class MyImviz(ConfigHelper):
    _default_configuration = config

im = MyImviz()
im.show()

In [ ]:
im.load_data(data)

In [ ]:
# this works cleanly since I defined a proper reference viewer in my configuration
im.app.add_data_to_viewer('image-viewer', data, ext='SCI')

In [ ]:
# it would be nice if some convenience for creating data parsers could live in the ConfigHelper or easily 
# decorate / attach to the new helper class